FEE TRANSACTION PREDICTION IN BLOCKS, Perederei Bohdan KM-03


NOTES
Потрібно подумати щодо останнього блоку, якщо блок тільки з'явився, то для генерації краще брати старий мемпул до генерації блоку, якщо вже певний час пройшов, то новий

Спочатку краще не заморочуватися, а потім вже додати можливість вибирати старий чи новий блок. Зараз краще зробити можелювання даного блоку, а час для збирання статистики для моделювання краще брати як відстань від останнього блоку

1) Preparing mempool

In [5]:
import requests
import json
import time

NODE_URL = 'https://responsive-frosty-tent.bcoin.discover.quiknode.pro/67427eeafaf13fedc399dc5b124246ffce77033a/'

response = requests.get("https://mempool.space/api/block/" + requests.get("https://mempool.space/api/blocks/tip/hash").text)
response = response.json()
LAST_BLOCK_TIME = response.get("timestamp")

MAX_AGE_FOR_TRANSACTION = (int(time.time()) - LAST_BLOCK_TIME) # In seconds

'''
Getting age for transcatcions
'''

class Transaction:
    def __init__(self, fee, size, time):
        self.fee = fee
        self.size = size
        self.time = time
    
    def __repr__(self):
        return f"Transaction({self.fee}, {self.size}, {self.time})"
    def __str__(self):
        return str(self.fee) + str(self.size) + str(self.time)

'''
Downloading mempool and saving it
'''
headers = {
        'content-type': 'application/json',
    }
data = '{"jsonrpc": "1.0", "id": "curltest", "method": "getrawmempool", "params": [true]}'
response = requests.post(NODE_URL, headers=headers, data=data)

with open('mempool' + '.json', 'w', encoding='utf-8') as f:
            json.dump(response.json(), f, ensure_ascii=False, indent=4)
mempool = json.load(open('mempool.json'))

'''
Importing mempool and sorting it
'''
mempool = list()
json_mempool = response.json().get('result')
for tx in json_mempool:
    tx = json_mempool.get(tx)
    mempool.append(Transaction(tx.get("fee"), tx.get("size"), tx.get("time")))
mempool.sort(key = lambda tx: tx.time, reverse=True)

'''
Building transaction set for random process
'''
transactionListForGeneration = list()
for tx in mempool:
    if tx.time < mempool[0].time - MAX_AGE_FOR_TRANSACTION:
        break
    transactionListForGeneration.append(tx)

print(len(mempool))
print(len(transactionListForGeneration))

3102
1700


2) Getting set for generating N number of transactions every BUCKET_TIME seconds

In [6]:
BUCKET_TIME = 5 # In seconds
NumberOfTransactionsEachInterval = list()

currentTransaction = 0
for tx in mempool:
    if tx.time < mempool[0].time - MAX_AGE_FOR_TRANSACTION:
        break
    
    if tx.time < mempool[0].time - (len(NumberOfTransactionsEachInterval) + 1)*BUCKET_TIME:
        NumberOfTransactionsEachInterval.append(currentTransaction)
        currentTransaction = 0
    
    currentTransaction += 1
print(NumberOfTransactionsEachInterval)
print(len(NumberOfTransactionsEachInterval))


[22, 5, 5, 5, 12, 11, 6, 7, 7, 18, 6, 6, 9, 13, 10, 4, 8, 10, 8, 8, 4, 6, 17, 10, 9, 6, 9, 3, 12, 9, 9, 16, 9, 1, 13, 15, 12, 1, 11, 5, 5, 11, 13, 5, 1, 9, 11, 16, 8, 12, 10, 16, 3, 8, 9, 7, 11, 2, 15, 10, 9, 8, 12, 8, 15, 2, 18, 8, 17, 3, 13, 3, 15, 27, 23, 1, 10, 4, 18, 8, 3, 15, 6, 16, 6, 10, 8, 3, 13, 11, 5, 8, 13, 6, 9, 18, 24, 11, 6, 6, 15, 5, 8, 7, 10, 7, 5, 18, 9, 5, 14, 7, 10, 11, 7, 24, 14, 1, 17, 14, 12, 15, 6, 11, 7, 6, 12, 4, 13, 5, 12, 10, 4, 26, 10, 22, 10, 2, 16, 12, 5, 4, 7, 14, 9, 18, 7, 12, 2, 11, 7, 13, 11, 7, 11, 6, 7, 17, 20, 14, 14, 3, 15, 7, 13, 10, 9, 6, 13, 12, 7, 5]
172


2) Getting probabilty of block appearing each second

In [7]:
import requests
import json
import random

URL_HASHRATE = "https://mempool.space/api/v1/mining/hashrate/3d"
URL_PROBABILITY = "https://blockchain.info/q/probability"

hashrate = int(requests.get(URL_HASHRATE).json().get("currentHashrate"))
mining_probabilty = float(requests.get(URL_PROBABILITY).text)
probabiltyOfBlockMining = hashrate * mining_probabilty # Every second
print(probabiltyOfBlockMining)


0.0018420445013144053


MODELING

In [8]:
import random
# Getting time of last block
# lastBlockTime = 
NUMBER_OF_BLOCKS = 5
TIME_INTERVAL_BLOCK_APPEARING = 5
MAX_BLOCK_SIZE = 1000000

MEMPOOL = mempool.copy()
TRANSACTIONS_SET = transactionListForGeneration.copy()
TRANSACTION_FREQUENCY_SET = NumberOfTransactionsEachInterval.copy()

class Block:
    def __init__(self, transactions):
        self.transactions = transactions
    

def getBlocksTimeAppearing():
    timeOfBlocks = list()
    probabilityEveryInterval = probabiltyOfBlockMining * TIME_INTERVAL_BLOCK_APPEARING
    for i in range(0, NUMBER_OF_BLOCKS):
        time = 0
        while probabilityEveryInterval < random.uniform(0, 1):
            time += TIME_INTERVAL_BLOCK_APPEARING
        timeOfBlocks.append(time)
    
    if timeOfBlocks[0] - MAX_AGE_FOR_TRANSACTION > 0:
         timeOfBlocks[0] -= MAX_AGE_FOR_TRANSACTION
    
    return timeOfBlocks

def runGeneration():
    MEMPOOL = mempool.copy()

    timeOfBlocksAppearing = getBlocksTimeAppearing()
    print(getBlocksTimeAppearing())
    blocks = list()

    for blockTime in timeOfBlocksAppearing:
        
        # Generating new transactions before block appears
        numberOfTransactionGenerations = int(blockTime / BUCKET_TIME)
        for generation in range(0, numberOfTransactionGenerations):

            for transaction in range(0, random.choice(TRANSACTION_FREQUENCY_SET)):
                tx = random.choice(TRANSACTIONS_SET)
                # TODO: calculate new time of transaction
                MEMPOOL.append(Transaction(tx.fee, tx.size, tx.time))

        # Sorting mempool
        MEMPOOL.sort(key=lambda tx: (tx.fee/tx.size, tx.size), reverse=True)

        current_size = 0
        txs = 0
        while current_size < MAX_BLOCK_SIZE and txs < len(MEMPOOL):
            current_size += MEMPOOL[txs].size
            txs += 1

        blocks.append(Block(MEMPOOL[:txs]))
        MEMPOOL = MEMPOOL[txs:]

    return blocks

for block in runGeneration():
    txs = block.transactions
    # Getting transaction with lowest fee in block
    print(txs[len(txs)-1:])
    print((txs[len(txs)-1].fee / txs[len(txs)-1].size) * 10**(8))

[105, 465, 835, 45, 145]
[Transaction(0.00020598, 9725, 1670826417)]
2.118046272493573
[Transaction(0.0003098, 28085, 1670823279)]
1.1030799359088481
[Transaction(2.23e-06, 403, 1670709351)]
0.553349875930521
[Transaction(1.66e-06, 166, 1670829077)]
1.0
[Transaction(1.33e-06, 133, 1670828583)]
1.0
